In [1]:
from os import environ
environ['CUDA_VISIBLE_DEVICES'] = '3'

In [269]:
import json
import unicodedata
from pprint import pprint
import numpy as np
from __future__ import print_function
from functools import reduce
import re
from nltk.tokenize import word_tokenize
from keras.utils.data_utils import get_file
import keras
from keras import layers
from keras.layers import recurrent, Bidirectional, Dense, Flatten, Dropout, LSTM, GRU, Conv1D, MaxPooling1D, UpSampling1D,BatchNormalization
from keras.layers import Activation
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K


from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from functools import partial
from itertools import product
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
embeddings_index = dict()
f = open('glove/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [341]:
with open('train.json') as data_file:    
    data = json.load(data_file)
ids = []
titles = dict()
contexts = list()
questions = list()
contexts_uni = dict()
questions_uni = dict()
answers_text = list()
answers_start = dict()
for i in range(len(data)):
    paragraphs = data[i]["paragraphs"]
    title = data[i]["title"]
    for j in range(len(paragraphs)):
        context = paragraphs[j]["context"]
        qas = paragraphs[j]["qas"]
        for k in range(len(qas)):
            id_ = qas[k]["id"]
            answer = qas[k]["answer"]
            question = qas[k]["question"]
            ids.append(id_)
            titles[id_] = title
            contexts.append(unicodedata.normalize('NFKD',context).encode('ascii', 'ignore'))
            contexts_uni[id_] = context
            answers_start[id_] = answer["answer_start"]
            answers_text.append(unicodedata.normalize('NFKD',answer["text"]).encode('ascii', 'ignore'))  
            questions.append(unicodedata.normalize('NFKD',question).encode('ascii', 'ignore'))
            questions_uni[id_] = question

max_para = 672
max_q = 50
dimension = 300
train_len = len(questions)
l = [i for i in range(len(ids))]
# l.sort(key=lambda x: np.random.random())
ind = l[:train_len]

In [175]:
t = Tokenizer()
t.fit_on_texts(contexts+questions) 
vocab_size = len(t.word_index) + 1
print(vocab_size)

72049


In [343]:
ans_seg = np.zeros((train_len, max_para))
ans_seg_cat = np.zeros((train_len, max_para, 3))
ans_se = np.zeros((train_len, 2))
ans_start = np.zeros((train_len, max_para))
ans_end = np.zeros((train_len, max_para))
#3 classes
# 0 for out of context
# 1 for background
# 2 for answers
for i in ind:
    cur_context = contexts[i]
    ans_st = answers_start[ids[i]]
    ans_text = t.texts_to_sequences([answers_text[i],answers_text[i]])
    encoded_context = t.texts_to_sequences([contexts[i][0:answers_start[ids[i]]],contexts[i]])
    ans_start[i, len(encoded_context[0])] = 1
    ans_end[i, len(encoded_context[0])+len(ans_text[0])] = 1


(61379, 672)

In [344]:
para_vocab_size = len(t.word_index) + 1
encoded_para = t.texts_to_sequences(contexts[0:train_len])
padded_para = pad_sequences(encoded_para, maxlen=max_para, padding='post')

In [345]:
embedding_para = zeros((para_vocab_size, dimension))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_para[i] = embedding_vector

In [346]:
qns_vocab_size = len(t.word_index) + 1
encoded_qns = t.texts_to_sequences(questions[0:train_len])
padded_qns = pad_sequences(encoded_qns, maxlen=max_q, padding='post')

In [347]:
embedding_qns = zeros((qns_vocab_size, dimension))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_qns[i] = embedding_vector

In [260]:
print(contexts[0])
print(len(max(encoded_para, key=len)))
print(len(questions))
print(len(max(encoded_qns, key=len)))
print(padded_para.shape)

In 1790, the first federal population census was taken in the United States. Enumerators were instructed to classify free residents as white or "other." Only the heads of households were identified by name in the federal census until 1850. Native Americans were included among "Other;" in later censuses, they were included as "Free people of color" if they were not living on Indian reservations. Slaves were counted separately from free persons in all the censuses until the Civil War and end of slavery. In later censuses, people of African descent were classified by appearance as mulatto (which recognized visible European ancestry in addition to African) or black.
107
61379
107
(500, 672)


In [233]:
def convBN_init(input_layer, conv_channels):
    convlayer = Conv1D(conv_channels, 5, padding = 'same')(input_layer)
    BN = BatchNormalization(axis=-1, momentum = 0.99, epsilon=0.001, center=True, scale = True, 
                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                            beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(convlayer)
    activation = layers.PReLU()(BN)

    return activation
def convBN(input_layer, conv_channels):
    convlayer = Conv1D(conv_channels, 3, padding = 'same')(input_layer)
    BN = BatchNormalization(axis=-1, momentum = 0.99, epsilon=0.001, center=True, scale = True, 
                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                            beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(convlayer)
    activation = layers.PReLU()(BN)

    return activation
def convBN_dilate(input_layer, conv_channels):
    convlayer = Conv1D(conv_channels, 3, padding = 'same', dilation_rate=2)(input_layer)
    BN = BatchNormalization(axis=-1, momentum = 0.99, epsilon=0.001, center=True, scale = True, 
                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                            beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(convlayer)
    activation = layers.PReLU()(BN)

    return activation
def RU_dilate(input_layer, conv_channels, d_rate):
    # input tensor for a 3-channel 256x256 image
    x = input_layer
    # 3x3 conv with 3 output channels (same as input channels)
    y = Conv1D(conv_channels, 3, padding='same', dilation_rate = d_rate)(x)
    y = Conv1D(conv_channels, 3, padding='same', dilation_rate = d_rate)(y)
    # this returns x + y.
    z = layers.add([x, y])
    return z
def RU(input_layer, conv_channels):
    # input tensor for a 3-channel 256x256 image
    x = input_layer
    # 3x3 conv with 3 output channels (same as input channels)
    y = Conv1D(conv_channels, 3, padding='same')(x)
    y = Conv1D(conv_channels, 3, padding='same')(y)
    # this returns x + y.
    z = layers.add([x, y])
    return z
#Full resolution residual unit
def FRRU(input_z, input_y, conv_channels):
    #Calculate pooling size
    zshape = input_z.shape.as_list()[1]
    yshape = input_y.shape.as_list()[1]
   
    ps = int(zshape / yshape)

    downsampling_z = MaxPooling1D(pool_size = ps, strides = None, padding ='valid')(input_z)
  
    concat_yz = keras.layers.concatenate([input_y, downsampling_z])
    y_stream = convBN(concat_yz, conv_channels)
    y_stream = convBN(y_stream, conv_channels)

    upsampling_y = UpSampling1D(size=ps)(y_stream)
    upsampling_y = Conv1D(input_z.shape.as_list()[2], 1, padding = 'same')(upsampling_y)
    z_stream = keras.layers.add([upsampling_y,input_z])
    return z_stream, y_stream
def FRRU_dilate(input_z, input_y, conv_channels):
    #Calculate pooling size
    zshape = input_z.shape.as_list()[1]
    yshape = input_y.shape.as_list()[1]
   
    ps = int(zshape / yshape)

    downsampling_z = MaxPooling1D(pool_size = ps, strides = None, padding ='valid')(input_z)
  
    concat_yz = keras.layers.concatenate([input_y, downsampling_z])
    y_stream = convBN_dilate(concat_yz, conv_channels)
    y_stream = convBN_dilate(y_stream, conv_channels)

    upsampling_y = UpSampling1D(size=ps)(y_stream)
    upsampling_y = Conv1D(input_z.shape.as_list()[2], 1, padding = 'same')(upsampling_y)
    z_stream = keras.layers.add([upsampling_y,input_z])
    return z_stream, y_stream

In [267]:
class WeightedCategoricalCrossEntropy(object):

    def __init__(self, weights):
        nb_cl = len(weights)
        self.weights = np.ones((nb_cl, nb_cl))
        for class_idx, class_weight in weights.items():
            self.weights[0][class_idx] = class_weight
            self.weights[class_idx][0] = class_weight
        self.__name__ = 'w_categorical_crossentropy'

    def __call__(self, y_true, y_pred):
        return self.w_categorical_crossentropy(y_true, y_pred)

    def w_categorical_crossentropy(self, y_true, y_pred):
        nb_cl = len(self.weights)
        final_mask = K.zeros_like(y_pred[..., 0])
        y_pred_max = K.max(y_pred, axis=-1)
        y_pred_max = K.expand_dims(y_pred_max, axis=-1)
        y_pred_max_mat = K.equal(y_pred, y_pred_max)
        for c_p, c_t in product(range(nb_cl), range(nb_cl)):
            w = K.cast(self.weights[c_t, c_p], K.floatx())
            y_p = K.cast(y_pred_max_mat[..., c_p], K.floatx())
            y_t = K.cast(y_pred_max_mat[..., c_t], K.floatx())
            final_mask += w * y_p * y_t
        return K.categorical_crossentropy(y_pred, y_true) * final_mask

In [348]:
## Encoding
input_para = Input(shape=(max_para,))
input_qns = Input(shape=(max_q,))

em_paras = Embedding(vocab_size, dimension, weights=[embedding_para], input_length=max_para, trainable = False)(input_para)
em_qns = Embedding(vocab_size, dimension, weights=[embedding_qns], input_length = max_q, trainable = False)(input_qns)

merge_1 = layers.dot([em_paras, em_qns], axes = 2 )
A_Q = layers.Activation("softmax")(merge_1)
merge_2 = layers.dot([em_qns, em_paras], axes = 2 )
A_D = layers.Activation("softmax")(merge_2)
C_Q = layers.dot([A_Q, em_paras], axes = 1 )
print(C_Q.shape)
C_Q = layers.concatenate([em_qns, C_Q], axis=2)
C_D = layers.dot([A_D, C_Q], axes=1)
C_ = layers.concatenate([em_paras, C_D], axis=2)
print(C_.shape)

#stream = Dropout(0.2)(input_layer)
stream = convBN_init(C_, 200)
stream = RU(stream, 200)
stream = RU(stream, 200)
# stream = RU(stream, 48)

stream_y = MaxPooling1D(pool_size = 2, strides = None, padding ='valid')(stream)
stream_z = Conv1D(200, 1, padding='same')(stream)

stream_z, stream_y = FRRU(stream_z, stream_y, 250)
stream_z, stream_y = FRRU(stream_z, stream_y, 250)
# stream_z, stream_y = FRRU(stream_z, stream_y, 400)
print(stream_y.shape.as_list())
stream_y = MaxPooling1D(pool_size = 2, strides = None, padding ='valid')(stream_y)

stream_z, stream_y = FRRU(stream_z, stream_y, 300)
stream_z, stream_y = FRRU_dilate(stream_z, stream_y, 300)

stream_y = Dropout(0.5)(stream_y)
# stream_z, stream_y = FRRU(stream_z, stream_y, 450)

# stream_z, stream_y = FRRU(stream_z, stream_y, 192)
print(stream_y.shape.as_list())
stream_y = MaxPooling1D(pool_size = 2, strides = None, padding ='valid')(stream_y)

# stream_z, stream_y = FRRU(stream_z, stream_y, 300)
stream_z, stream_y = FRRU(stream_z, stream_y, 300)
stream_z, stream_y = FRRU_dilate(stream_z, stream_y, 300)
stream_y = Dropout(0.5)(stream_y)
stream_z = Dropout(0.5)(stream_z)
# stream_z, stream_y = FRRU(stream_z, stream_y, 384)
print(stream_y.shape.as_list())
stream_y = MaxPooling1D(pool_size = 2, strides = None, padding ='valid')(stream_y)
stream_z, stream_y = FRRU(stream_z, stream_y, 384)
# stream_z, stream_y = FRRU(stream_z, stream_y, 384)
print(stream_y.shape.as_list())
stream_y = UpSampling1D(size=2)(stream_y)
stream_z, stream_y = FRRU(stream_z, stream_y, 300)
stream_z, stream_y = FRRU(stream_z, stream_y, 300)
print(stream_y.shape.as_list())
stream_y = UpSampling1D(size=2)(stream_y)
stream_z, stream_y = FRRU(stream_z, stream_y, 250)
stream_y = Dropout(0.5)(stream_y)
stream_z, stream_y = FRRU(stream_z, stream_y, 250)

print(stream_y.shape.as_list())
stream_y = UpSampling1D(size=2)(stream_y)
stream_z, stream_y = FRRU(stream_z, stream_y, 200)
stream_y = Dropout(0.5)(stream_y)
stream_z, stream_y = FRRU(stream_z, stream_y, 200)
print(stream_y.shape.as_list())
stream_y = UpSampling1D(size=2)(stream_y)
print(stream_y.shape.as_list())
print(stream_z.shape.as_list())
stream = keras.layers.concatenate([stream_y, stream_z])
stream = Conv1D(100, 1, padding='same')(stream)
stream = Dropout(0.5)(stream)
print(stream.shape.as_list())
# stream = RU(stream, 48)
stream = RU(stream, 100)

stream = RU(stream, 100)

start = Conv1D(1, 1, padding='same')(stream)
start = Flatten()(start)
start = Activation('softmax', name = 'output_1')(start)
end = Conv1D(1, 1, padding='same')(stream)
end = Flatten()(end)
end = Activation('softmax', name = 'output_2')(end)

(?, 50, 300)
(?, 672, 900)
[None, 336, 250]
[None, 168, 300]
[None, 84, 300]
[None, 42, 384]
[None, 84, 300]
[None, 168, 250]
[None, 336, 200]
[None, 672, 200]
[None, 672, 200]
[None, 672, 100]


In [349]:
model = Model([input_para, input_qns],[start, end])
model.compile(optimizer='adam',
              loss={'output_1': 'categorical_crossentropy', 'output_2': 'categorical_crossentropy'},
              metrics=['accuracy'])

In [350]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_21 (InputLayer)            (None, 672)           0                                            
____________________________________________________________________________________________________
input_22 (InputLayer)            (None, 50)            0                                            
____________________________________________________________________________________________________
embedding_15 (Embedding)         (None, 672, 300)      21614700                                     
____________________________________________________________________________________________________
embedding_16 (Embedding)         (None, 50, 300)       21614700                                     
___________________________________________________________________________________________

In [265]:
from keras.utils import plot_model
plot_model(model, to_file='nlp.png')

In [ ]:
model.fit([padded_para, padded_qns], [ans_start, ans_end],
          batch_size=64,
          epochs=10,
          validation_split=0.15)

Train on 52172 samples, validate on 9207 samples
Epoch 1/10
52172/52172 [==============================] - 448s - loss: 6.8219 - output_1_loss: 3.4275 - output_2_loss: 3.3943 - output_1_acc: 0.1904 - output_2_acc: 0.1819 - val_loss: 7.5715 - val_output_1_loss: 3.8634 - val_output_2_loss: 3.7081 - val_output_1_acc: 0.2259 - val_output_2_acc: 0.2319
Epoch 2/10
 3712/52172 [=>............................] - ETA: 390s - loss: 6.1558 - output_1_loss: 3.1179 - output_2_loss: 3.0379 - output_1_acc: 0.2540 - output_2_acc: 0.2484

In [ ]:
model.save('NLP_first.h5')

In [322]:
predict = model.predict([padded_para, padded_qns])

In [340]:
c = 0
for i in range(train_len):
    if np.argmax(predict[0][i]) == np.argmax(ans_start[i]) and np.argmax(predict[1][i])== np.argmax(ans_end[i]):
        c = c + 1
print(c*1.0/train_len)

0.686


In [281]:
index_word = {v: k for k, v in t.word_index.items()} # map back
seqs = t.texts_to_sequences(contexts[0])
words = []
for seq in seqs:
    if len(seq):
        words.append(index_word.get(seq[0]))
    else:
        words.append(' ')
print(''.join(words)) # output

in 1790  the first federal population census was taken in the united states  enumerators were instructed to classify free residents as white or  other   only the heads of households were identified by name in the federal census until 1850  native americans were included among  other   in later censuses  they were included as  free people of color  if they were not living on indian reservations  slaves were counted separately from free persons in all the censuses until the civil war and end of slavery  in later censuses  people of african descent were classified by appearance as mulatto  which recognized visible european ancestry in addition to african  or black 
